# Fill test data glosses/examples with LLMs

In [1]:
import pandas as pd
import json
import os
import openai
from dotenv import load_dotenv
import time
import re
import unicodedata
import numpy as np
pd.options.mode.chained_assignment = None  # default='warn'

## 1. Load the data and compute the number of requests to make

In [1]:
FILE_TO_READ = './data/test/axolotl.test.ru.tsv'
AUGMENTED_EXAMPLES = 3

language = FILE_TO_READ.split('.')[-2]
filename = FILE_TO_READ.split('/')[-1].split('.')[0:-1]
filename = '.'.join(filename)
language, filename

('ru', 'axolotl.test.ru')

The following requets will be done:
1. For each glos-known row that doesn't have an example available, an example will be generated (+3)**
2. For each gloss-unknown row, a gloss is generated. This is also useful for achieving directly subtask 2 (+1)
3. For each gloss-known row, at least three examples are needed, so if a sense has only one example, two more are generated (+2)

** To avoid repeating requests, the generation of step 3. is also done in step 1. directly for non-example rows.

In [3]:
df = pd.read_csv(FILE_TO_READ, sep='\t')
df.info()
print("\nColumns to fill 'example':" , df['example'].isna().sum(), "(output ×3)")
print("Columns to fill 'gloss':" , df['gloss'].isna().sum(), "(output ×1)")
print("Columns to augment:" , df[~df['gloss'].isna()].shape[0]-df['example'].isna().sum(), "(output ×2)")
print("Total:", df['example'].isna().sum() + df['gloss'].isna().sum() + df[~df['gloss'].isna()].shape[0], "requests")

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2126 entries, 0 to 2125
Data columns (total 9 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   usage_id              2126 non-null   object 
 1   word                  2126 non-null   object 
 2   orth                  2126 non-null   object 
 3   sense_id              424 non-null    object 
 4   gloss                 424 non-null    object 
 5   example               1990 non-null   object 
 6   indices_target_token  0 non-null      float64
 7   date                  2126 non-null   object 
 8   period                2126 non-null   object 
dtypes: float64(1), object(8)
memory usage: 149.6+ KB

Columns to fill 'example': 136 (output ×3)
Columns to fill 'gloss': 1702 (output ×1)
Columns to augment: 288 (output ×2)
Total: 2262 requests


## 2. Define methods to send and store requests to OpenAI's API

In [4]:
load_dotenv('./gpt35.env')
openai.api_type = "azure"
openai.api_version = os.getenv("OPENAI_API_VERSION")
api_key = os.getenv("OPENAI_API_KEY")
api_base = os.getenv("OPENAI_ENDPOINT")
engine=os.getenv("OPENAI_IMPLEMENTATION")
model=os.getenv("MODEL_NAME")
model

'gpt-3.5-turbo'

In [5]:
MAX_ITERATIONS = 1 # only one repetition is allowed
def request(prompt, max_tokens=500, temperature=0, iterations=0):
    """Request a completion from the OpenAI API.
    :param prompt: The prompt to send to the API
    :param max_tokens: The maximum number of tokens in the output
    :param temperature: The degree of randomness in the output
    :return response: The response from the API
    :return usage: The count of token usage from the API { "input", "output" }
    """
    try:
        response = openai.ChatCompletion.create(
            api_key=api_key,
            api_base=api_base,
            engine=engine,
            max_tokens=max_tokens,
            temperature=temperature,
            messages=[{"role": "user", "content": prompt}],
        )
        finish_reason = response["choices"][0]["finish_reason"]
        rsp = "" if finish_reason == "content_filter" else response["choices"][0]["message"]["content"]
        return rsp, { "input": response["usage"]["prompt_tokens"], "output": response["usage"]["completion_tokens"] }, finish_reason
    except Exception as e:
        response = ""
        usage = {"input": 0, "output": 0}
        if f"{e}".startswith("The response was filtered due to the prompt triggering Azure OpenAI's content management policy"):
            finish_reason = "content_filter"
        else:
            if ("Max retries exceeded with url" in f"{e}") and (iterations < MAX_ITERATIONS):
                    print("MAX_RETRIES triggered. Retrying in 60 seconds...")
                    time.sleep(60)
                    return request(prompt, max_tokens, temperature, iterations+1) # retry after 60 seconds
            finish_reason = f"ERROR [{type(e).__name__}]: {e}"
        return response, usage, finish_reason

gen_prompt = lambda prompt, *args: prompt.format(*args)

Finally, a method to find the target word within the sentence (wether the "word" or the "orth"):

In [6]:
def generate_substrings(word):
    substrings = []
    for i in range(len(word), 0, -1):
        substrings.append(word[:i])
    return substrings[1:-1]

def find_sub_list(sl,l): # not used because some examples have no exact coincidence
    sll=len(sl)
    for ind in (i for i,e in enumerate(l) if e==sl[0]):
        if l[ind:ind+sll]==sl:
            return ind+1,ind+sll # +1 for the [CLS] token

def extract_letters(input_string):
    return re.sub(r'[^а-яА-Яa-zA-ZÀ-ÿёЁ\u0300-\u036f-]', '', input_string)

def remove_accents(word):
    normalized_word = unicodedata.normalize('NFD', word)
    cleaned_word = re.sub(r'[\u0300-\u036f]', '', normalized_word)
    return cleaned_word

def find_word_containing_target(sentence, target_word):
    index = sentence.find(target_word)
    if index == -1:
        return None
    start_index = sentence.rfind(" ", 0, index) + 1 if index != 0 else 0
    end_index = sentence.find(" ", index + len(target_word)) if sentence.find(" ", index + len(target_word)) != -1 else len(sentence)
    final_char = final_char = " " if target_word.endswith(" ") else ""
    return sentence[start_index:end_index].split()[0] + final_char

def get_search(example, word, orth=None, print_search=False):
    # append the words to search in the example, in the desired ORDER
    # 1 - the word (with an ending character), and the word itself
    search = [f"{word} ", f"{word},", f"{word}.", word]

    # 2 - the orthographic form (with an ending character), and the orthographic form itself
    if orth and orth != word:
        search += [f"{orth} ", f"{orth},", f"{orth}.", orth]

    # 3 - all substrings of the word (i.e. выходить -> ['выходит', 'выходи', 'выход', 'выхо', 'вых', 'вы'])
    search += generate_substrings(word)

    # 4 - all substrings of the orthographic form
    if orth:
        search.extend([i for i in generate_substrings(orth) if i not in search])

    # 5 - the word without accents (with an ending character), and the word without accents itself
    unicoded_word = remove_accents(word)
    if unicoded_word != word:
        search += [f"{unicoded_word} ", f"{unicoded_word},", f"{unicoded_word}.", unicoded_word]
    
    # 6 - the orthographic form without accents (with an ending character), and the orthographic form without accents itself
    if orth and orth != word:
        unicoded_orth = remove_accents(orth)
        if unicoded_orth != orth:
            search += [f"{unicoded_orth} ", f"{unicoded_orth},", f"{unicoded_orth}.", unicoded_orth]

    # 7 - all substrings of the word without accents
    if unicoded_word != word:
        search.extend([i for i in generate_substrings(unicoded_word) if i not in search])

    # 8 - all substrings of the orthographic form without accents
    if orth and orth != word and unicoded_orth != orth:
        search.extend([i for i in generate_substrings(unicoded_orth) if i not in search])

    if print_search:
        print(f"Searching for: {search}")

    # FIND the first search-string that is within the example, if any (in upper or lowercase)
    for s in search:
        search_word = find_word_containing_target(example, s)
        if search_word:
            break
        search_word = find_word_containing_target(example.lower(), s.lower())
        if search_word:
            index = example.lower().find(search_word)
            if index == -1:
                # this should never happen
                raise Exception(f"Found '{search_word}' in '{example.lower()}', but then not found...")
            else:
                search_word = example[index:index + len(search_word)]
            break
    else:
        search_word = ""
    return extract_letters(search_word)

In [7]:
print(f'Find {(df.loc[0, "word"], df.loc[0, "orth"])} in the sentence: {df.loc[0, "example"]}')
result_example = get_search(df.loc[0, "example"], df.loc[0, "word"], df.loc[0, "orth"], print_search=True)
print(f"Found: {result_example}" if result_example else "Not found")

Find ('мёрзлый', 'мёрзлый') in the sentence: Мерзлая земля скажется, только руку приложи От того нельзя, что земля мерзла. Мерзлой
Searching for: ['мёрзлый ', 'мёрзлый,', 'мёрзлый.', 'мёрзлый', 'мёрзлы', 'мёрзл', 'мёрз', 'мёр', 'мё', 'мерзлыи ', 'мерзлыи,', 'мерзлыи.', 'мерзлыи', 'мерзлы', 'мерзл', 'мерз', 'мер', 'ме']
Found: мерзла


## 3. Prepare and send requests for each of the 3 cases (2 in the same)

### 1. For each glos-known row that doesn't have an example available, an example will be generated (+1)
&&
### 3. For each gloss-known row, two more examples will be generated (data augmentation) (+2)
Additionally, in this step, the data is augmented: an extra-2 examples are generated for each request (3 in total)

#### 1.1. Prepare dataset and prompt

In [8]:
df[~df['gloss'].isna()][['word', 'orth', 'sense_id', 'gloss', 'example']]

,word,orth,sense_id,gloss,example
0,мёрзлый,мёрзлый,merzlyj_khrrgztit4U,"разг., устар. очень холодный; ледяной","Мерзлая земля скажется, только руку приложи От..."
6,мёрзлый,мёрзлый,merzlyj_g74j6SCAoAc,покрытый ледяной коркой; обледенелый,NaN
15,элемент,элементъ,element_mip-D10cqu0,"хим. простое вещество, атомный состав которого...",NaN
30,кабак,кабакъ,kabak_hO7-f2Xxaic,"перен., разг. место, заведение, где продают и ...",Кабаком называли собст. откупное питейное заве...
38,трубач,трубачъ,trubach_Y8MimbC7EPc,птица псофия,NaN
...,...,...,...,...,...
2099,хлыст,хлыстъ,hlyst_0S-PQCiOfWU,"спец. ствол дерева, отделённый от комля и очищ...",Хлыст в 15 сажен.
2100,слыхать,слыхать,slyhat'_UUkfjifb7-I,"более относится к вестям, слухам, что слышат, ...",Что слыхать?
2117,скорлупа,скорлупа,skorlupa_Oq5fheu46Q0,"твёрдый наружный слой яиц, орехов и т. п.; твё...","Скорлупа ореховая, миндальная. Рачья скролупа...."
2120,чирикать,чирикать,chirikat'_h05Xmk8q6z4,"издавать высокие частые звуки, характерные для...",Воробьи ватажатся и чирикают на дождь.


In [9]:
# original (english): 'Give me ONLY {} line-separated usage examples of the word "{}" with the following definition:\n{}'

if language == "ru":
    prompt = 'Приведите мне ТОЛЬКО {} примера употребления слова "{}" в строках со следующим определением:\n{}'
elif language == "fi":
    prompt = 'Anna minulle VAIN {} rivieroteltua esimerkkiä sanasta "{}" seuraavalla määritelmällä:\n{}'
else: # language == "de"
    prompt = 'Geben Sie mir NUR {} durch Zeilen getrennte Verwendungsbeispiele für das Wort „{}“ mit der folgenden Definition:\n{}'

exp_fill = df[~df['gloss'].isna()]
for i in range(5):
    print(prompt.format(AUGMENTED_EXAMPLES, exp_fill.iloc[i, 1], exp_fill.iloc[i, 4]), end="\n\n")
del exp_fill

Приведите мне ТОЛЬКО 3 примера употребления слова "мёрзлый" в строках со следующим определением:
разг., устар. очень холодный; ледяной

Приведите мне ТОЛЬКО 3 примера употребления слова "мёрзлый" в строках со следующим определением:
покрытый ледяной коркой; обледенелый

Приведите мне ТОЛЬКО 3 примера употребления слова "элемент" в строках со следующим определением:
хим. простое вещество, атомный состав которого однороден

Приведите мне ТОЛЬКО 3 примера употребления слова "кабак" в строках со следующим определением:
перен., разг. место, заведение, где продают и распивают спиртные напитки; бар, ресторан, пивная, рюмочная и т. п.

Приведите мне ТОЛЬКО 3 примера употребления слова "трубач" в строках со следующим определением:
птица псофия



#### 1.2. Send requests

In [10]:
def generate_examples(n, prompt):
    #response, usage, finish_reason = request(prompt)
    return ["hello" for _ in range(n)]

In [31]:
mod_df = df.copy()
id_template = '_'.join(df.loc[0, 'usage_id'].split("_")[:2]) + '_'
last_id = int(df.loc[len(df)-1, 'usage_id'].split("_")[-1])

index = 0

In [32]:
while index < len(df):
    row = mod_df.iloc[index]
    gloss = row['gloss']
    if f"{gloss}" != "nan":
        sense_id = row['sense_id']
        word = row['word']
        example = row['example']

        found = len(mod_df[(mod_df['sense_id'] == sense_id) & (~mod_df['example'].isna())])
        n = AUGMENTED_EXAMPLES - found
        if n > 0:
            p = gen_prompt(prompt, n, word, gloss)
            examples = generate_examples(n, p)
            if f"{example}" == "nan":
                row['example'] = examples[0]
                mod_df.iloc[index] = row
                start_at = 1
            else:
                start_at = 0
            for e in examples[start_at:]:
                last_id += 1
                row['example'] = e
                row['usage_id'] = f"{id_template}_{last_id}"
                mod_df.loc[len(mod_df)] = row
        else:
            print(f"No examples needed for {word} ({sense_id}), found {found}")
    index += 1

mod_df

No examples needed for мёрзлый (merzlyj_khrrgztit4U), found 3
No examples needed for мёрзлый (merzlyj_g74j6SCAoAc), found 3
No examples needed for элемент (element_mip-D10cqu0), found 3
No examples needed for кабак (kabak_hO7-f2Xxaic), found 3
No examples needed for трубач (trubach_Y8MimbC7EPc), found 3
No examples needed for трубач (trubach_nssqD8fAIms), found 3
No examples needed for трубач (trubach_UkbiF9TOfxA), found 3
No examples needed for трубач (trubach_wxZia0Vo9uc), found 3
No examples needed for славный (slavnyj_QejLMPEGfKI), found 3
No examples needed for славный (slavnyj_Fz9qx2T7iEw), found 3
No examples needed for центр (tsentr_dyP_fnTKHNc), found 3
No examples needed for центр (tsentr_fg4SnH95ui0), found 3
No examples needed for установка (ustanovka_AXgSofwcvRE), found 3
No examples needed for символ (simvol_8yUMO8v3gTg), found 3
No examples needed for символ (simvol_9VHw-Ae7JMI), found 3
No examples needed for мелькать (mel'kat'_KUwcKbpT9Pc), found 3
No examples needed f

No examples needed for убрать (ubrat'_xA9v_vrwH9k), found 3
No examples needed for убрать (ubrat'_D9UcvPrPNvo), found 3
No examples needed for убрать (ubrat'_ZN_KfE7HJwE), found 3
No examples needed for убрать (ubrat'_EU4WwNCNaWc), found 3
No examples needed for таблетка (tabletka_4WCOQV_TVow), found 3
No examples needed for кстати (kstati_LyTmhLiHa-c), found 3
No examples needed for катран (katran_mzukG75NpjU), found 3
No examples needed for катран (katran_LOdTKPg5LZQ), found 3
No examples needed for катран (katran_4kLXvqL2iUA), found 3
No examples needed for меч (mech_8BzMXhucg3o), found 3
No examples needed for меч (mech_hpf6pgruCPE), found 3
No examples needed for меч (mech_ZKJwgWJHkmI), found 3
No examples needed for точить (tochit'_ms4ocSMq-rI), found 3
No examples needed for точить (tochit'_S0j_TlhmnyI), found 3
No examples needed for точить (tochit'_mMfkUwcK-60), found 3
No examples needed for точить (tochit'_rud1Rqukz0E), found 3
No examples needed for точить (tochit'_fkDFUfq0

,usage_id,word,orth,sense_id,gloss,example,indices_target_token,date,period
0,test_ru_0,мёрзлый,мёрзлый,merzlyj_khrrgztit4U,"разг., устар. очень холодный; ледяной","Мерзлая земля скажется, только руку приложи От...",NaN,old,old
1,test_ru_1,мёрзлый,мёрзлый,NaN,NaN,Пантелей Прокофьевич сунул Григорию мёрзлую ру...,NaN,new,new
2,test_ru_2,мёрзлый,мёрзлый,NaN,NaN,"Между домом и рельсами, за широкой мёрзлой луж...",NaN,new,new
3,test_ru_3,мёрзлый,мёрзлый,NaN,NaN,"— Да, холодно, должно быть. На полу мёрзлые та...",NaN,new,new
4,test_ru_4,мёрзлый,мёрзлый,NaN,NaN,"Когда мосты были прорваны, безоружные солдаты,...",NaN,new,new
...,...,...,...,...,...,...,...,...,...
2948,test_ru__2952,скорлупа,скорлупа,skorlupa_Oq5fheu46Q0,"твёрдый наружный слой яиц, орехов и т. п.; твё...",hello,NaN,old,old
2949,test_ru__2953,чирикать,чирикать,chirikat'_h05Xmk8q6z4,"издавать высокие частые звуки, характерные для...",hello,NaN,old,old
2950,test_ru__2954,чирикать,чирикать,chirikat'_h05Xmk8q6z4,"издавать высокие частые звуки, характерные для...",hello,NaN,old,old
2951,test_ru__2955,чирикать,чирикать,chirikat'_LlwmrNjmWWQ,"Чирикнуть кого, ряз. слово из Рязанской губерн...",hello,NaN,old,old


In [12]:
mod_df.to_csv(f'./data/augmented/prev.{filename}.tsv', sep='\t', index=False)

### 2. For each gloss-unknown row, a gloss is generated. This is also useful for achieving directly subtask 2 (+1)

#### 2.1. Prepare dataset and prompt

In [13]:
df_2 = mod_df.copy()
df_2[df_2['gloss'].isna()][['word', 'orth', 'sense_id', 'gloss', 'example']]

,word,orth,sense_id,gloss,example
1,мёрзлый,мёрзлый,NaN,NaN,Пантелей Прокофьевич сунул Григорию мёрзлую ру...
2,мёрзлый,мёрзлый,NaN,NaN,"Между домом и рельсами, за широкой мёрзлой луж..."
3,мёрзлый,мёрзлый,NaN,NaN,"— Да, холодно, должно быть. На полу мёрзлые та..."
4,мёрзлый,мёрзлый,NaN,NaN,"Когда мосты были прорваны, безоружные солдаты,..."
5,мёрзлый,мёрзлый,NaN,NaN,Мёрзлый пар валил с загнанных насмерть лошадей...
...,...,...,...,...,...
2119,чирикать,чирикать,NaN,NaN,"Канарейка, немилосердно трещавшая целый день, ..."
2121,чирикать,чирикать,NaN,NaN,"Очарованные этой, как говорится, оргией красок..."
2123,чирикать,чирикать,NaN,NaN,"— Ладно, барышни, — произнёс я, поднимаясь. — ..."
2124,чирикать,чирикать,NaN,NaN,"— Ты, я знаю, в Ленинград собрался. Мой тебе с..."


In [14]:
# original (english): 'What does "{}" mean in the following examples (ONLY return JSON with the key "definition" - detailed definition)?\n{}'

if language == "ru":
    prompt = 'Что означает «{}» в следующих примерах (ТОЛЬКО возвращайте JSON с ключом «определение» — подробное определение)?\n{}'
elif language == "fi":
    prompt = 'Mitä «{}» tarkoittaa seuraavissa esimerkeissä (palauta VAIN JSON avaimella "definition" - monisanainen määritelmä)?\n{}'
else: # language == "de"
    prompt = 'Was bedeutet „{}“ in den folgenden Beispielen (geben sie JSON NUR mit dem Schlüssel „definition“ zurück – ausführliche Definition)?\n{}'

gls_fill = df_2[df_2['gloss'].isna()]
for i in range(5):
    print(prompt.format(gls_fill.iloc[i, 1], gls_fill.iloc[i, 5]), end="\n\n")
del gls_fill

Что означает «мёрзлый» в следующих примерах (ТОЛЬКО возвращайте JSON с ключом «определение» — подробное определение)?
Пантелей Прокофьевич сунул Григорию мёрзлую руку, сел на край лавки, запахивая полу тулупа, обходя взглядом Аксинью, пристывшую у люльки.

Что означает «мёрзлый» в следующих примерах (ТОЛЬКО возвращайте JSON с ключом «определение» — подробное определение)?
Между домом и рельсами, за широкой мёрзлой лужей, проходила дорога, по которой возили дрова и воду.

Что означает «мёрзлый» в следующих примерах (ТОЛЬКО возвращайте JSON с ключом «определение» — подробное определение)?
— Да, холодно, должно быть. На полу мёрзлые тараканы валяются. И мыши тоже помёрзли.

Что означает «мёрзлый» в следующих примерах (ТОЛЬКО возвращайте JSON с ключом «определение» — подробное определение)?
Когда мосты были прорваны, безоружные солдаты, московские жители, женщины с детьми, бывшие в обозе французов, — всё под влиянием силы инерции не сдавалось, а бежало вперед в лодки, в мёрзлую воду.

Что 

#### 2.2. Send requests

In [15]:
def generate_gloss(prompt):
    #response, usage, finish_reason = request(prompt)
    return "gloss"

In [30]:
mod_df_2 = df_2.copy()

In [29]:
for index, row in mod_df_2.iterrows():
    gloss = row["gloss"]
    if f"{gloss}" == "nan":
        example = row["example"]
        word = row["word"]
        orth = row["orth"]
        s = get_search(example, word, orth)
        p = gen_prompt(prompt, s, example)
        gloss = generate_gloss(p)
        row['gloss'] = gloss
        mod_df_2.iloc[index] = row
        break
mod_df_2

,usage_id,word,orth,sense_id,gloss,example,indices_target_token,date,period
0,test_ru_0,мёрзлый,мёрзлый,merzlyj_khrrgztit4U,"разг., устар. очень холодный; ледяной","Мерзлая земля скажется, только руку приложи От...",NaN,old,old
1,test_ru_1,мёрзлый,мёрзлый,NaN,gloss,Пантелей Прокофьевич сунул Григорию мёрзлую ру...,NaN,new,new
2,test_ru_2,мёрзлый,мёрзлый,NaN,NaN,"Между домом и рельсами, за широкой мёрзлой луж...",NaN,new,new
3,test_ru_3,мёрзлый,мёрзлый,NaN,NaN,"— Да, холодно, должно быть. На полу мёрзлые та...",NaN,new,new
4,test_ru_4,мёрзлый,мёрзлый,NaN,NaN,"Когда мосты были прорваны, безоружные солдаты,...",NaN,new,new
...,...,...,...,...,...,...,...,...,...
2948,test_ru__2952,скорлупа,скорлупа,skorlupa_Oq5fheu46Q0,"твёрдый наружный слой яиц, орехов и т. п.; твё...",hello,NaN,old,old
2949,test_ru__2953,чирикать,чирикать,chirikat'_h05Xmk8q6z4,"издавать высокие частые звуки, характерные для...",hello,NaN,old,old
2950,test_ru__2954,чирикать,чирикать,chirikat'_h05Xmk8q6z4,"издавать высокие частые звуки, характерные для...",hello,NaN,old,old
2951,test_ru__2955,чирикать,чирикать,chirikat'_LlwmrNjmWWQ,"Чирикнуть кого, ряз. слово из Рязанской губерн...",hello,NaN,old,old


In [ ]:
mod_df_2.to_csv(f'./data/augmented/{filename}.tsv', sep='\t', index=False)